# SQL and RDBMS

 * Data is generally stored in databases, rather than in flat files
     * Reduced redundancy
     * More consistent
     * Better backups!
     * Data entry/storage/retrieval is more efficient

There are many types of databases!

 * Sometimes, data is stored in tabular formats
 * Sometimes as documents (hierarchical)
 * Sometimes as a graph (network-based)

Today, we will look at Relational Database Management Systems (RDBMS)

![](./assets/relational_model.png)

## Characteristics of a Relational Database

 * Data are stored as tables (Rows and Columns)
 * All values are scalar (each row/column entry has exactly 1 value)
 * Each column has exactly 1 type (numeric, text, etc.)
 * Tables have Key columns, which are used to index the table
 * A Primary key is a column (or set of columns) that *uniquely* identifies a row in a table.
     * Must be unique
     * cannot be NULL
 * A Foreign key is a column whose value is required to match the primary key of another table

## Benefits of the Relational Model

 * Data is easy to retrieve and query
 * Flexible (easy to add/delete tables)
 * Reduced redundancy
 

## Disadvantages
 * Sometimes slow and difficult to scale
 * Not ideal for storing hierarchical data
 * Must adhere to a fixed schema (bad for unstructured data)

# Structured Query Language (SQL)

 * A language used to query data (and more!) from relational databases
 * Many different flavors depending on the database:
     * Oracle
     * Microsoft SQL
     * MySQL
     * etc.

## Database Tables

 * Every table has a name 
 * Contains records (rows)

In [1]:
import sqlite3 # library for working with sqlite database
conn = sqlite3.connect("./data/diabetes.db") # Create a connection to the on-disk database

In [2]:
import pandas as pd

#### Example Query

In [3]:
pd.read_sql("SELECT * FROM patient LIMIT 25", conn)

,index,patient_nbr,race,gender
0,0,8222157,Caucasian,Female
1,1,55629189,Caucasian,Female
2,2,86047875,AfricanAmerican,Female
3,3,82442376,Caucasian,Male
4,4,42519267,Caucasian,Male
5,5,82637451,Caucasian,Male
6,6,84259809,Caucasian,Male
7,7,114882984,Caucasian,Male
8,8,48330783,Caucasian,Female
9,9,63555939,Caucasian,Female


In [4]:
pd.read_sql("SELECT * FROM sqlite_master where type='table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,patient,patient,2,"CREATE TABLE ""patient"" (\n""index"" INTEGER,\n ..."
1,table,diagnosis,diagnosis,796,"CREATE TABLE ""diagnosis"" (\n""index"" INTEGER,\n..."
2,table,medications,medications,1888,"CREATE TABLE ""medications"" (\n""index"" INTEGER,..."
3,table,encounter,encounter,4317,"CREATE TABLE ""encounter"" (\n""index"" INTEGER,\n..."
4,table,ccs_crosswalk,ccs_crosswalk,6308,"CREATE TABLE ""ccs_crosswalk"" (\n""index"" INTEGE..."


### SELECT clause

```
SELECT column_name1, column_name2 
FROM table1```

In [5]:
pd.read_sql("SELECT race, gender FROM patient", conn)

,race,gender
0,Caucasian,Female
1,Caucasian,Female
2,AfricanAmerican,Female
3,Caucasian,Male
4,Caucasian,Male
5,Caucasian,Male
6,Caucasian,Male
7,Caucasian,Male
8,Caucasian,Female
9,Caucasian,Female


#### SELECT DISTINCT

returns unique values

In [6]:
pd.read_sql("SELECT DISTINCT race, gender FROM patient", conn)

,race,gender
0,Caucasian,Female
1,AfricanAmerican,Female
2,Caucasian,Male
3,AfricanAmerican,Male
4,None,Male
5,None,Female
6,Other,Female
7,Other,Male
8,Asian,Male
9,Hispanic,Female


### WHERE Clauses

Where clauses allow you to *filter* data in your SQL query. There are many logical operators that you can use with the WHERE clase. Here is a simple one:

In [7]:
pd.read_sql("SELECT DISTINCT race, gender FROM patient WHERE race = 'Caucasian'", conn)

,race,gender
0,Caucasian,Female
1,Caucasian,Male


In [8]:
pd.read_sql("SELECT patient_nbr FROM patient WHERE patient_nbr = 77586282", conn)

,patient_nbr
0,77586282


Here is a list of some operators you can use in your WHERE clause. 

![](./assets/WHEREclause.png)

#### BETWEEN

In [9]:
pd.read_sql("SELECT * FROM patient WHERE patient_nbr BETWEEN 10000 AND 99999", conn)

,index,patient_nbr,race,gender
0,222,48573,AfricanAmerican,Female
1,230,91305,Caucasian,Male
2,265,15849,Caucasian,Male
3,266,25434,Caucasian,Male
4,285,42867,Caucasian,Male
5,297,78246,Caucasian,Female
6,317,69921,Caucasian,Male
7,318,12447,Caucasian,Female
8,375,20295,Caucasian,Male
9,393,70110,Caucasian,Female


#### LIKE

The `LIKE` operator lets you specify matches in text, much like regular expressions. However, it is considerably less powerful. The keys to know are the `%` and `_` operators.

`%` represents 0, one, or many characters (Wildcard)

`_` represents 1 character (Any)

In [10]:
pd.read_sql("SELECT DISTINCT race, gender FROM patient WHERE gender LIKE '%male'", conn)

,race,gender
0,Caucasian,Female
1,AfricanAmerican,Female
2,Caucasian,Male
3,AfricanAmerican,Male
4,None,Male
5,None,Female
6,Other,Female
7,Other,Male
8,Asian,Male
9,Hispanic,Female


In [12]:
pd.read_sql("SELECT DISTINCT race, gender FROM patient WHERE gender LIKE '__male'", conn)

,race,gender
0,Caucasian,Female
1,AfricanAmerican,Female
2,None,Female
3,Other,Female
4,Hispanic,Female
5,Asian,Female


#### IN 

In [13]:
pd.read_sql("SELECT DISTINCT race, gender FROM patient WHERE race in ('Caucasian', 'Hispanic')", conn)

,race,gender
0,Caucasian,Female
1,Caucasian,Male
2,Hispanic,Female
3,Hispanic,Male


### LOGIC (AND/OR/NOT)

As you might imagine, you can create complex WHERE clauses by using the `AND`, `OR`, and `NOT` keywords. In addition, you can wrap the subclauses in parentheses to make sure that they execute together.

In [14]:
pd.read_sql("""SELECT DISTINCT race, gender 
                FROM patient 
                WHERE race in ('Caucasian', 'Hispanic')
                    AND gender = 'Female'
            """, conn)


,race,gender
0,Caucasian,Female
1,Hispanic,Female


In [16]:
pd.read_sql("""SELECT DISTINCT race, gender 
                    FROM patient 
                    WHERE race IN ('Caucasian', 'Hispanic') OR gender ='Female'
                    """, conn)

,race,gender
0,Caucasian,Female
1,AfricanAmerican,Female
2,Caucasian,Male
3,None,Female
4,Other,Female
5,Hispanic,Female
6,Hispanic,Male
7,Asian,Female


In [17]:
pd.read_sql("""SELECT DISTINCT race, gender 
                FROM patient 
                WHERE race in ('Caucasian', 'Hispanic')
                    OR NOT gender = 'Female'
            """, conn)

,race,gender
0,Caucasian,Female
1,Caucasian,Male
2,AfricanAmerican,Male
3,None,Male
4,Other,Male
5,Asian,Male
6,Hispanic,Female
7,Hispanic,Male


In [18]:
pd.read_sql("""SELECT DISTINCT race, gender
                FROM patient
                WHERE gender = 'Female' AND (race = 'Other' OR race = 'Asian')
""", conn)

,race,gender
0,Other,Female
1,Asian,Female


### ORDER BY

In [19]:
pd.read_sql("""SELECT patient_nbr, gender
                FROM patient
                ORDER BY patient_nbr
""", conn)

,patient_nbr,gender
0,135,Female
1,378,Female
2,729,Female
3,774,Female
4,927,Female
5,1152,Female
6,1305,Female
7,1314,Female
8,1629,Male
9,2025,Female


In [20]:
pd.read_sql("""SELECT patient_nbr, gender
                FROM patient
                ORDER BY patient_nbr DESC
""", conn)

,patient_nbr,gender
0,189502619,Male
1,189481478,Female
2,189445127,Female
3,189365864,Male
4,189351095,Female
5,189349430,Female
6,189332087,Male
7,189298877,Female
8,189257846,Male
9,189215762,Male


### NULL values

In [21]:
pd.read_sql("""SELECT patient_nbr, gender
                FROM patient
                WHERE gender IS NULL
""", conn)

,patient_nbr,gender
0,100695042,None
1,60524946,None
2,78119847,None


In [22]:
pd.read_sql("""SELECT patient_nbr, gender
                FROM patient
                WHERE gender = NULL
""", conn)

,patient_nbr,gender


In [23]:
pd.read_sql("""SELECT patient_nbr, gender
                FROM patient
                WHERE gender IS NOT NULL
""", conn)

,patient_nbr,gender
0,8222157,Female
1,55629189,Female
2,86047875,Female
3,82442376,Male
4,42519267,Male
5,82637451,Male
6,84259809,Male
7,114882984,Male
8,48330783,Female
9,63555939,Female


### Min, Max, Count, Avg, Sum

SQL also has some built-in functions for summarizing data. For example, you can call `MIN(column_name)` and it will return the minimum of a column in a select statement

In [24]:
pd.read_sql("""SELECT MIN(num_medications), MAX(num_medications), AVG(num_medications)
                FROM encounter
""", conn)

,MIN(num_medications),MAX(num_medications),AVG(num_medications)
0,1,81,16.021844


### Aliases

Sometimes, SQL queries contain long table names, or column names, and it is easier to refer to them by another name, or alias. In addition, derived columns like those returned from MIN(), MAX(), etc. often look better when reformatted.

##### Column name alias

In [25]:
pd.read_sql("""SELECT MIN(num_medications) AS Minimum_medications, MAX(num_medications), AVG(num_medications)
                FROM encounter
""", conn)

,Minimum_medications,MAX(num_medications),AVG(num_medications)
0,1,81,16.021844


# Joining Tables

The power of relational databases comes from their relation structure, which enables the user to merge tables together in order to combine information across tables.

In [26]:
pd.read_sql("""SELECT *
                FROM diagnosis
                LIMIT 10
""", conn)

,index,encounter_id,diag_1,diag_2,diag_3
0,0,2278392,25083,None,None
1,1,149190,2760,25001,2550
2,2,64410,6480,2500,V270
3,3,500364,0080,25043,4030
4,4,16680,1970,1570,2500
5,5,35754,4140,4110,2500
6,6,55842,4140,4110,V450
7,7,63768,4280,4920,2500
8,8,12522,3980,4270,0380
9,9,15738,4340,1980,4860


In [29]:
pd.read_sql("""SELECT "ICD-9-CM CODE"
                FROM ccs_crosswalk
                LIMIT 10""", conn)

,ICD-9-CM CODE
0,
1,01000
2,01001
3,01002
4,01003
5,01004
6,01005
7,01006
8,01010
9,01011


### 3 Important Types of Joins:

### Inner Join

![](./assets/innerjoin.png)

As the figure above shows, an inner join takes only the values of the join columns that are in BOTH tables and returns the result

In [30]:
pd.read_sql("""SELECT *
                FROM diagnosis
                INNER JOIN ccs_crosswalk ON diagnosis.diag_1 = ccs_crosswalk."ICD-9-CM CODE"
""", conn)

,index,encounter_id,diag_1,diag_2,diag_3,index,ICD-9-CM CODE,CCS CATEGORY,CCS CATEGORY DESCRIPTION,ICD-9-CM CODE DESCRIPTION,OPTIONAL CCS CATEGORY,OPTIONAL CCS CATEGORY DESCRIPTION
0,0,2278392,25083,None,None,2445,25083,50,DiabMel w/cm,DIAB W MANIF NEC TYPE I DM UNCONT (Begin 1993),,
1,1,149190,2760,25001,2550,2630,2760,55,Fluid/elc dx,HYPEROSMOLALITY,,
2,3,500364,0080,25043,4030,5447,0080,135,Intest infct,E. COLI ENTERITIS (Begin 1980 End 1992),,
3,4,16680,1970,1570,2500,2080,1970,42,2ndary malig,SECONDARY MALIG NEO LUNG,,
4,5,35754,4140,4110,2500,4681,4140,101,Coron athero,CORONARY ATHEROSCLEROSIS (End 1994),,
5,6,55842,4140,4110,V450,4681,4140,101,Coron athero,CORONARY ATHEROSCLEROSIS (End 1994),,
6,7,63768,4280,4920,2500,4771,4280,108,chf;nonhp,CONGESTIVE HEART FAILURE,,
7,8,12522,3980,4270,0380,4559,3980,97,Carditis,RHEUMATIC MYOCARDITIS,,
8,9,15738,4340,1980,4860,4801,4340,109,Acute CVD,CEREBRAL THROMBOSIS (End 1993),,
9,11,36900,1570,2880,1970,1375,1570,17,Pancreas can,MAL NEO PANCREAS HEAD,,


### Left Join

![](./assets/leftjoin.png)

In [31]:
pd.read_sql("""SELECT *
                FROM diagnosis
                LEFT JOIN ccs_crosswalk ON diagnosis.diag_1 = ccs_crosswalk."ICD-9-CM CODE"
""", conn)

,index,encounter_id,diag_1,diag_2,diag_3,index,ICD-9-CM CODE,CCS CATEGORY,CCS CATEGORY DESCRIPTION,ICD-9-CM CODE DESCRIPTION,OPTIONAL CCS CATEGORY,OPTIONAL CCS CATEGORY DESCRIPTION
0,0,2278392,25083,None,None,2445.0,25083,50,DiabMel w/cm,DIAB W MANIF NEC TYPE I DM UNCONT (Begin 1993),,
1,1,149190,2760,25001,2550,2630.0,2760,55,Fluid/elc dx,HYPEROSMOLALITY,,
2,2,64410,6480,2500,V270,NaN,None,None,None,None,None,None
3,3,500364,0080,25043,4030,5447.0,0080,135,Intest infct,E. COLI ENTERITIS (Begin 1980 End 1992),,
4,4,16680,1970,1570,2500,2080.0,1970,42,2ndary malig,SECONDARY MALIG NEO LUNG,,
5,5,35754,4140,4110,2500,4681.0,4140,101,Coron athero,CORONARY ATHEROSCLEROSIS (End 1994),,
6,6,55842,4140,4110,V450,4681.0,4140,101,Coron athero,CORONARY ATHEROSCLEROSIS (End 1994),,
7,7,63768,4280,4920,2500,4771.0,4280,108,chf;nonhp,CONGESTIVE HEART FAILURE,,
8,8,12522,3980,4270,0380,4559.0,3980,97,Carditis,RHEUMATIC MYOCARDITIS,,
9,9,15738,4340,1980,4860,4801.0,4340,109,Acute CVD,CEREBRAL THROMBOSIS (End 1993),,


### Full Join (Full outer join)

![](./assets/fulljoin.png)

In [33]:
pd.read_sql("""SELECT *
                FROM diagnosis
                FULL JOIN ccs_crosswalk ON diagnosis.diag_1 = ccs_crosswalk."ICD-9-CM CODE"
""", conn)

DatabaseError: Execution failed on sql 'SELECT *
                FROM diagnosis
                FULL JOIN ccs_crosswalk ON diagnosis.diag_1 = ccs_crosswalk."ICD-9-CM CODE"
': RIGHT and FULL OUTER JOINs are not currently supported

### Group By:
Just like in Pandas, we have group by functionality in SQL as well.

In [35]:
pd.read_sql("""SELECT cx."CCS CATEGORY DESCRIPTION", COUNT(cx."CCS CATEGORY DESCRIPTION")
                FROM diagnosis dx
                INNER JOIN ccs_crosswalk cx ON dx.diag_1 = cx."ICD-9-CM CODE"
                GROUP BY cx."CCS CATEGORY DESCRIPTION"
""", conn)

,CCS CATEGORY DESCRIPTION,"COUNT(cx.""CCS CATEGORY DESCRIPTION"")"
0,2ndary malig,587
1,Abdomnl pain,561
2,Acq foot def,2
3,Acute CVD,2120
4,Acute MI,3614
5,Adjustment disorders,37
6,Alcohol-related disorders,370
7,Allergy,115
8,Anal/rectal,298
9,Anemia,861


### Having

In [36]:
pd.read_sql("""SELECT ccs_crosswalk."CCS CATEGORY DESCRIPTION", COUNT(ccs_crosswalk."CCS CATEGORY DESCRIPTION") 
                FROM diagnosis
                INNER JOIN ccs_crosswalk ON diagnosis.diag_1 = ccs_crosswalk."ICD-9-CM CODE"
                GROUP BY ccs_crosswalk."CCS CATEGORY DESCRIPTION"
                HAVING COUNT(ccs_crosswalk."CCS CATEGORY DESCRIPTION") > 1000
""", conn)

,CCS CATEGORY DESCRIPTION,"COUNT(ccs_crosswalk.""CCS CATEGORY DESCRIPTION"")"
0,Acute CVD,2120
1,Acute MI,3614
2,Back problem,1026
3,COPD,2331
4,Coma/brn dmg,2019
5,Complic proc,1318
6,Coron athero,6952
7,DiabMel w/cm,3998
8,Dysrhythmia,2867
9,Fluid/elc dx,1889


### Case Statements

In [37]:
pd.read_sql("""SELECT num_medications, 
                (CASE 
                    WHEN num_medications > 5 THEN "Greater than 5 medications"
                    WHEN num_medications <= 5 THEN "Less than or equal to 5 medications"
                END) AS GreaterThan5
                FROM encounter
""", conn)

,num_medications,GreaterThan5
0,1,Less than or equal to 5 medications
1,18,Greater than 5 medications
2,13,Greater than 5 medications
3,16,Greater than 5 medications
4,8,Greater than 5 medications
5,16,Greater than 5 medications
6,21,Greater than 5 medications
7,12,Greater than 5 medications
8,28,Greater than 5 medications
9,18,Greater than 5 medications


## Execution Order (https://sqlbolt.com/lesson/select_queries_order_of_execution)

Under the hood, 

### 1. FROM and JOINs
The FROM clause, and subsequent JOINs are first executed to determine the total working set of data that is being queried. This includes subqueries in this clause, and can cause temporary tables to be created under the hood containing all the columns and rows of the tables being joined.

### 2. WHERE
Once we have the total working set of data, the first-pass WHERE constraints are applied to the individual rows, and rows that do not satisfy the constraint are discarded. Each of the constraints can only access columns directly from the tables requested in the FROM clause. Aliases in the SELECT part of the query are not accessible in most databases since they may include expressions dependent on parts of the query that have not yet executed.

### 3. GROUP BY
The remaining rows after the WHERE constraints are applied are then grouped based on common values in the column specified in the GROUP BY clause. As a result of the grouping, there will only be as many rows as there are unique values in that column. Implicitly, this means that you should only need to use this when you have aggregate functions in your query.

### 4. HAVING
If the query has a GROUP BY clause, then the constraints in the HAVING clause are then applied to the grouped rows, discard the grouped rows that don't satisfy the constraint. Like the WHERE clause, aliases are also not accessible from this step in most databases.

### 5. SELECT
Any expressions in the SELECT part of the query are finally computed.

### 6. DISTINCT
Of the remaining rows, rows with duplicate values in the column marked as DISTINCT will be discarded.

### 7. ORDER BY
If an order is specified by the ORDER BY clause, the rows are then sorted by the specified data in either ascending or descending order. Since all the expressions in the SELECT part of the query have been computed, you can reference aliases in this clause.

### 8. LIMIT / OFFSET
Finally, the rows that fall outside the range specified by the LIMIT and OFFSET are discarded, leaving the final set of rows to be returned from the query.

### And much, much more...

### Resources:
[Interactive SQL book](https://selectstarsql.com/)

[Quick Reference](https://www.w3schools.com/sql/sql_quickref.asp)

## In-class exercises:

### Answer the following questions from HW2 with SQL statements instead of Pandas (answers my differ slightly due to preprocessing)

**How many unique encounters are there? How many unique patients?**

In [41]:
pd.read_sql("""SELECT COUNT(DISTINCT encounter_id) AS unique_encounters
                FROM encounter
""", conn)

,unique_encounters
0,101766


**What is the most amount of encounters that a single patient has in the dataset?**

In [48]:
pd.read_sql("""SELECT COUNT(encounter_id) Number_of_encounters 
                FROM encounter
                GROUP BY patient_nbr
                ORDER BY Number_of_encounters DESC
                LIMIT 1
""", conn)

,Number_of_encounters
0,40


In [49]:
pd.read_sql("""SELECT MAX(inner.num_encounters)
            FROM (SELECT COUNT(encounter_id)
                    AS num_encounters
                    FROM encounter
                    GROUP BY patient_nbr) AS inner
""", conn)

,MAX(inner.num_encounters)
0,40


**What is the average number of labs administered by race?**

## JOIN encounter with patient

In [53]:
pd.read_sql("""SELECT * FROM encounter LIMIT 5
""", conn).columns

Index(['index', 'encounter_id', 'patient_nbr', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'change', 'readmitted'],
      dtype='object')

In [55]:
pd.read_sql("""SELECT AVG(num_lab_procedures) AS average_labs, pat.race
                FROM encounter enc
                    INNER JOIN patient pat ON enc.patient_nbr = pat.patient_nbr
                GROUP BY race
""", conn)

,average_labs,race
0,44.190029,None
1,44.069732,AfricanAmerican
2,41.802343,Asian
3,42.835792,Caucasian
4,42.841210,Hispanic
5,43.776284,Other


**Create a new column that has the value of 1 if the medical specialty in that row contains the word Surgery and 0 otherwise** 

For this question, you can use the `LIKE` operator and `CASE` statements [Case Statements](https://www.w3schools.com/sql/sql_case.asp)

In [58]:
pd.read_sql("""SELECT enc.medical_specialty, 
                (CASE
                    WHEN enc.medical_specialty LIKE '%Surgery%' THEN 1
                    WHEN enc.medical_specialty NOT LIKE '%Surgery%' THEN 0
                END) AS is_surgery
                FROM encounter enc
                LIMIT 50
""", conn)

,medical_specialty,is_surgery
0,Pediatrics-Endocrinology,0.0
1,None,NaN
2,None,NaN
3,None,NaN
4,None,NaN
5,None,NaN
6,None,NaN
7,None,NaN
8,None,NaN
9,InternalMedicine,0.0
